In [1]:
import pandas as pd
import os
from utils_kg_search import get_all_observation,get_api_df
from call_chatgpt_with_zny_asyncio import get_gpt4api_df
from utils_data_format_conversion import convert_csv_to_sft

### csv格式转为jsonl格式


In [3]:
data = {
'id':['test-1'],
'turn_id':[1],
'source':['测试'],
'user-query':['世界的第三高峰是哪个'],
'Thought':['涉及事实问答，查询世界第三高峰的信息。'],
'API':['<|api_start|><|kvs|>APINAME=>QASearch<|kve|><|kvs|>CATEGORY=>地理<|kve|><|kvs|>QUERY=>世界第三高峰<|kve|><|kvs|>TAG=>世界&第三高峰<|kve|><|api_end|><|api_start|><|kvs|>APINAME=>QASearch<|kve|><|kvs|>CATEGORY=>地理<|kve|><|kvs|>QUERY=>世界第三高峰<|kve|><|kvs|>TAG=>世界&第三高峰<|kve|><|api_end|>'],
'observation':["[['一些内容','一些内容','一些内容']]"],
'assistant':['世界的第三高峰是干城章嘉峰，位于喜马拉雅山脉，海拔高达8586米。'],
'relevant_label':['相关']
}
df = pd.DataFrame(data)
sft_df = convert_csv_to_sft(df.copy(),api_flag=True)


### query生成基于API的回复

In [5]:
def get_prompts_df(df: pd.DataFrame, oneshot_prompt: str) -> pd.DataFrame:
    df['prompts'] = df.apply(lambda row: oneshot_prompt + f"""
    ---
    Background:
    ```{row['observation']}```
    Answer：
    """, axis=1)
    
    return df

In [8]:
querys = {'user-query':['介绍下刘翔','姚明有多高','世界上最高的山峰是什么？']}
df = pd.DataFrame(querys)

# step 1. 生成API
api_df = get_api_df(df,category='QASearch') # category 具体设置 见函数说明
api_df = api_df.astype(str)

# step 2. 获取observation 
obs_df = get_all_observation(api_df)

# step 3. 调用gpt
oneshot_prompt = '根据参考资料回答问题' # 这部分prompt可以根据自己的需求自定义
prompt_df = get_prompts_df(obs_df,oneshot_prompt)
gpt4_df = get_gpt4api_df(prompt_df, message_type='', max_request_times=1, 
                         qps=2, max_concurrent=5, asyncio_flag=False) # 各参数见函数说明 asyncio_flag=False 支持jupyter运行，=True速度更快，支持terminal运行

# step 4. 转成jsonl文件
gpt4_df = gpt4_df.astype(str)
gpt4_df['id'] = gpt4_df.index.astype(str)
gpt4_df['source'] = '测试'
sft_df = convert_csv_to_sft(gpt4_df,api_flag=True)
sft_df.to_json('out.jsonl', orient='records', lines=True, force_ascii=False)